In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

import sys
sys.path.append('../')

from utils.results import plot_class_distribution

In [ ]:
with open('../data/land_cover_representation/metadata.csv') as f:
    metadata = pd.read_csv(f)

classes = pd.unique(metadata['land_cover'])
class_distribution = pd.Series(metadata['land_cover']).value_counts()
classes_dict = pd.Series.to_dict(class_distribution)

class_doc = dict(zip(pd.unique(metadata['y']), pd.unique(metadata['land_cover'])))
class_doc = dict(sorted(class_doc.items()))
display(class_doc)

class_count = {}
for i in range(0, len(class_doc)):
    class_count[i] = (class_doc[i], classes_dict[class_doc[i]])

count_df = pd.DataFrame.from_dict(class_count, orient='index', columns=['land_cover', 'count'])

plot_class_distribution(count_df, title='Unbalanced Class Distribution')

In [ ]:
with open('../data/land_cover_representation/metadata_balanced.csv') as f:
    metadata2 = pd.read_csv(f)

classes2 = pd.unique(metadata2['land_cover'])
class_distribution2 = pd.Series(metadata2['land_cover']).value_counts()
classes_dict2 = pd.Series.to_dict(class_distribution2)

class_doc2 = dict(zip(pd.unique(metadata2['y']), pd.unique(metadata2['land_cover'])))
class_doc2 = dict(sorted(class_doc2.items()))

class_count2 = {}
for i in range(0, len(class_doc2)):
    class_count2[i] = (class_doc2[i], classes_dict2[class_doc2[i]])

count_df2 = pd.DataFrame.from_dict(class_count2, orient='index', columns=['land_cover', 'count'])

plot_class_distribution(count_df2, title='Modified Class Distribution')

In [ ]:
def group_classes_by_semantics(low_counts_df, high_count_classes, threshold=0.5):
    """
    Groups low-count classes into semantic groups based on similarity to high-count classes.
    """
    # Load a pre-trained model for semantic similarity (e.g., SentenceTransformer)
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Generate embeddings for high-count class names
    high_count_embeddings = model.encode(high_count_classes, convert_to_tensor=True)

    # Initialize a dictionary for low-count to high-count mappings
    low_to_high_mapping = {}

    for low_class in low_counts_df['land_cover']:
        # Generate embedding for the low-count class
        low_embedding = model.encode(low_class, convert_to_tensor=True)

        # Calculate cosine similarity with high-count classes
        similarities = util.cos_sim(low_embedding, high_count_embeddings).squeeze(0)
        
        # Find the high-count class with the highest similarity above the threshold
        max_sim_idx = similarities.argmax().item()
        if similarities[max_sim_idx] >= threshold:
            low_to_high_mapping[low_class] = high_count_classes[max_sim_idx]
        else:
            # Map to 'Other' if no high similarity is found
            low_to_high_mapping[low_class] = 'Other'

    return low_to_high_mapping

min_count = 2000
similarity_threshold = 0.5

class_distribution = pd.Series(metadata['land_cover']).value_counts()
classes_dict = pd.Series.to_dict(class_distribution)

class_doc = dict(zip(pd.unique(metadata['y']), pd.unique(metadata['land_cover'])))
class_doc = dict(sorted(class_doc.items()))

class_count = {}
for i in range(0, 61):
    class_count[i] = (class_doc[i], classes_dict[class_doc[i]])

count_df = pd.DataFrame.from_dict(class_count, orient='index', columns=['land_cover', 'count'])

low_counts_df = count_df[count_df['count'] < min_count]
high_count_df = count_df[count_df['count'] >= min_count]
high_count_classes = high_count_df['land_cover'].tolist()

low_to_high_mapping = group_classes_by_semantics(low_counts_df, high_count_classes, threshold=similarity_threshold)

display(low_to_high_mapping)